In [1]:
!pip install -q transformers


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(torch_device)


In [4]:
# encode context the generation is conditioned on
model_inputs = tokenizer('I enjoy walking with my cute dog', return_tensors='pt').to(torch_device)

# generate 40 new tokens
greedy_output = model.generate(**model_inputs, max_new_tokens=40)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure


In [45]:
# activate beam search and early_stopping
sample_outputs = model.generate(
    **model_inputs,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.5,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog. I'm not sure if he likes it or not. I'm sure he's going to love it. I'm sure he's going to love it. I'm sure he's going to love it. I'm sure he's going to love it. I'm sure he's going to love it. I'm sure he's going to love it. I'm sure he's going to love it. I'm sure he's going to love it. I'm sure he's going to love
1: I enjoy walking with my cute dog and I love to see him run and play. I love to play with my dog and I love to see him run and play.

I love to play with my dog and I love to see him run and play. I love to play with my dog and I love to see him run and play.

I love to play with my dog and I love to see him run and play. I love to play with my dog and I love to see him run and play.

2: I enjoy walking with my cute dog, but I'm a little concerned about the fact that he's going to be a lot older. I'm just hoping that he's not 